In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

RootDIR = "/home/jw3514/Work/ASD_Circuits_CellType/" # put this in the right place
os.chdir(RootDIR + "/notebooks_mouse_str") # put this in the right place
print(f"Current working directory: {os.getcwd()}")
sys.path.insert(1, RootDIR + 'src')
# Need to add src directory to Python path first

#sys.path.append("../src")
from ASD_Circuits import *

In [ ]:
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

str2reg = STR2Region()


In [ ]:
ExpZ2Mat = pd.read_csv("../dat/allen-mouse-exp/AllenMouseBrain_Z2bias.csv", index_col=0)

In [ ]:
def CountMut(DF):
    N_LGD, N_mis, N_Dmis, N_syn = 0,0,0,0
    for i, row in DF.iterrows():
        GeneEff = row["GeneEff"].split(";")[0]
        if GeneEff in ["frameshift", "splice_acceptor", "splice_donor", "start_lost", "stop_gained", "stop_lost"]:
            N_LGD += 1
        elif GeneEff == "missense":
            N_mis += 1
            row["REVEL"] = row["REVEL"].split(";")[0]
            if row["REVEL"] != ".":
                if float(row["REVEL"]) > 0.5:
                    N_Dmis += 1
        elif GeneEff == "synonymous":
            N_syn += 1
    return N_LGD, N_mis, N_Dmis, N_syn
def Mut2GeneDF(MutDF, PPVs = [0.554, 0.333, 0.138, 0.130], LGD=True, Dmis=True):
    genes = np.array(list(set(MutDF["HGNC"].values)))
    dat = []
    gene2MutN = {}
    for g in genes:
        try:
            Entrez = int(GeneSymbol2Entrez[g])
        except:
            Entrez = -1
            continue
        Muts = MutDF[MutDF["HGNC"]==g]
        try:
            pLI = float(Muts["ExACpLI"].values[0])
        except:
            pLI = 0
        N_LGD, N_Mis, N_Dmis, N_Syn = CountMut(Muts)
        if not LGD:
            N_LGD = 0
        if not Dmis:
            N_Dmis = 0
        if pLI > 0.5:
            gene2MutN[Entrez] = N_LGD * PPVs[0] + N_Dmis * PPVs[1]
        else:
            gene2MutN[Entrez] = N_LGD * PPVs[2] + N_Dmis * PPVs[3]
    return gene2MutN
def GetBiasDat(DF, STRs):
    biases = DF.loc[STRs, "EFFECT"].values
    return np.mean(biases)
def GetBootsDist(DFs, STRs):
    dat = []
    for df in DFs:
        tmp = []
        biases = df.loc[STRs, "EFFECT"].values
        dat.append(np.mean(biases))
    return np.array(dat)
def GetBootErr(DFs1, DFs2, STRs):
    dat = []
    for df1, df2 in zip(DFs1, DFs2):
        tmp = []
        biases1 = df1.loc[STRs, "EFFECT"].values
        biases2 = df2.loc[STRs, "EFFECT"].values
        bias_diff = biases1 - biases2
        dat.append(np.mean(bias_diff))
    return np.array(dat)
def Filt_LGD_Mis(DF, Dmis=True):
    dat= []
    for i, row in DF.iterrows():
        GeneEff = row["GeneEff"].split(";")[0]
        if GeneEff in ["frameshift", "splice_acceptor", "splice_donor", "start_lost", "stop_gained", "stop_lost"]:
            dat.append(row.values)
        elif GeneEff == "missense":
            if Dmis:
                if GeneEff == "missense":
                    row["REVEL"] = row["REVEL"].split(";")[0]
                    if row["REVEL"] != ".":
                        if float(row["REVEL"]) > 0.5:
                            dat.append(row.values)
            else:
                if GeneEff == "missense":
                    dat.append(row.values)
    return pd.DataFrame(dat, columns = DF.columns.values)

# Create Master Table for Phenotype

# IQ and MutType

In [ ]:
ASC_IQ_dat = pd.read_excel("../dat/Genetics/1-s2.0-S0092867419313984-mmc4.xlsx", sheet_name="Phenotype") # This is SuppTab from ASC paper
ASC_IQ_dat = ASC_IQ_dat[ASC_IQ_dat["Role"]=="Proband"]
ASC_IQ_dat = ASC_IQ_dat.dropna(subset=['IQ'])

ASC_HIQ = ASC_IQ_dat[ASC_IQ_dat["IQ"]>70]["Phenotype_ID"].values
ASC_LIQ = ASC_IQ_dat[ASC_IQ_dat["IQ"]<=70]["Phenotype_ID"].values

Spark_IQ_dat = pd.read_csv("/home/jw3514/Work/ASD_Phenotype/Old/SPARK_Collection_Version6/core_descriptive_variables.csv")
Spark_IQ_dat = Spark_IQ_dat[Spark_IQ_dat["asd"]==True]
Spark_IQ_dat = Spark_IQ_dat.dropna(subset=['fsiq'])

Spark_HIQ = Spark_IQ_dat[Spark_IQ_dat["fsiq"]>70]["subject_sp_id"].values
Spark_LIQ = Spark_IQ_dat[Spark_IQ_dat["fsiq"]<=70]["subject_sp_id"].values

HighIQ = np.concatenate([ASC_HIQ, Spark_HIQ])
LowIQ = np.concatenate([ASC_LIQ, Spark_LIQ])

ASD_Discov_Muts = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/genes/SPARK/ASD_Discov_DNVs.txt", delimiter="\t")
ASD_Discov_Trios = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/genes/SPARK/ASD_Discov_Trios.txt", delimiter="\t")

ASD_Rep_Muts = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/genes/SPARK/ASD_Rep_DNVs.txt", delimiter="\t")
ASD_Rep_Trios = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/genes/SPARK/ASD_Rep_Trios.txt", delimiter="\t")

ASD_Muts = pd.concat([ASD_Discov_Muts, ASD_Rep_Muts])
ASD_Trios = pd.concat([ASD_Discov_Trios, ASD_Rep_Trios])

Spark_Meta_2stage = pd.read_excel("/home/jw3514/Work/ASD_Circuits/dat/genes/asd/TabS_DenovoWEST_Stage1+2.xlsx",
                           skiprows=2, sheet_name="TopDnEnrich")
Spark_Meta_HC = Spark_Meta_2stage[Spark_Meta_2stage["pDenovoWEST_Meta"]<=1.3e-6]
#Spark_Meta_HC = Spark_Meta_2stage[Spark_Meta_2stage["pDenovoWEST_Meta"]<=1e-3]
HighConfGenes = Spark_Meta_HC["HGNC"].values
HighConfMuts = ASD_Muts[ASD_Muts["HGNC"].isin(HighConfGenes)]
HighConfMuts = Filt_LGD_Mis(HighConfMuts, Dmis=True)


HIQ_Muts = HighConfMuts[HighConfMuts["IID"].isin(HighIQ)]
LIQ_Muts = HighConfMuts[HighConfMuts["IID"].isin(LowIQ)]

In [ ]:
print(HIQ_Muts.shape, LIQ_Muts.shape)

In [ ]:
# Calculate mutation counts and overlaps for visualization
import matplotlib.pyplot as plt
import matplotlib_venn as venn
import numpy as np
import pandas as pd
import seaborn as sns

# Get unique mutation identifiers (assuming mutations can be identified by IID + gene combination)
# Or use a unique mutation ID if available
HIQ_unique = set(HIQ_Muts.index) if hasattr(HIQ_Muts, 'index') else set(range(len(HIQ_Muts)))
LIQ_unique = set(LIQ_Muts.index) if hasattr(LIQ_Muts, 'index') else set(range(len(LIQ_Muts)))

# Better approach: use IID to identify which mutations belong to which group
HIQ_IIDs = set(HIQ_Muts["IID"].values) if "IID" in HIQ_Muts.columns else set()
LIQ_IIDs = set(LIQ_Muts["IID"].values) if "IID" in LIQ_Muts.columns else set()

# Count mutations by group
n_hiq = len(HIQ_Muts)
n_liq = len(LIQ_Muts)
n_total = len(HighConfMuts)

# Find mutations without IQ data (in HighConfMuts but not in HIQ or LIQ)
if "IID" in HighConfMuts.columns:
    all_iids_with_iq = HIQ_IIDs.union(LIQ_IIDs)
    muts_without_iq = HighConfMuts[~HighConfMuts["IID"].isin(all_iids_with_iq)]
    n_no_iq = len(muts_without_iq)
else:
    n_no_iq = n_total - n_hiq - n_liq

# Check for overlap (mutations that appear in both HIQ and LIQ - shouldn't happen but let's check)
if "IID" in HIQ_Muts.columns and "IID" in LIQ_Muts.columns:
    # Create mutation identifiers (IID + HGNC) to find overlaps
    HIQ_mut_ids = set(zip(HIQ_Muts["IID"].values, HIQ_Muts["HGNC"].values))
    LIQ_mut_ids = set(zip(LIQ_Muts["IID"].values, LIQ_Muts["HGNC"].values))
    overlap = HIQ_mut_ids.intersection(LIQ_mut_ids)
    n_overlap = len(overlap)
else:
    n_overlap = 0

print(f"Total High Confidence Mutations: {n_total}")
print(f"High IQ mutations: {n_hiq}")
print(f"Low IQ mutations: {n_liq}")
print(f"Mutations without IQ data: {n_no_iq}")
print(f"Overlap (should be 0): {n_overlap}")


In [ ]:
# # Create Venn Diagram
# fig, ax = plt.subplots(figsize=(10, 8), dpi=150)

# Prepare data for Venn diagram
# Since we have 3 groups (HIQ, LIQ, No IQ), we'll create a 3-circle Venn diagram
HIQ_set = set(zip(HIQ_Muts["IID"].values, HIQ_Muts["HGNC"].values)) if "IID" in HIQ_Muts.columns else set(range(len(HIQ_Muts)))
LIQ_set = set(zip(LIQ_Muts["IID"].values, LIQ_Muts["HGNC"].values)) if "IID" in LIQ_Muts.columns else set(range(len(LIQ_Muts)))

if "IID" in HighConfMuts.columns:
    all_mut_ids = set(zip(HighConfMuts["IID"].values, HighConfMuts["HGNC"].values))
    no_iq_set = all_mut_ids - HIQ_set - LIQ_set
else:
    no_iq_set = set()

# # Create Venn diagram
# venn_diagram = venn.venn3(
#     [HIQ_set, LIQ_set, no_iq_set],
#     set_labels=('High IQ\n(IQ > 70)', 'Low IQ\n(IQ ≤ 70)', 'No IQ Data'),
#     ax=ax
# )

# # Customize colors
# if venn_diagram:
#     for patch in venn_diagram.get_patch_ids():
#         if patch == '100':  # HIQ only
#             venn_diagram.get_patch_by_id('100').set_color('#AED6F1')
#             venn_diagram.get_patch_by_id('100').set_alpha(0.7)
#         elif patch == '010':  # LIQ only
#             venn_diagram.get_patch_by_id('010').set_color('#21618C')
#             venn_diagram.get_patch_by_id('010').set_alpha(0.7)
#         elif patch == '001':  # No IQ only
#             venn_diagram.get_patch_by_id('001').set_color('#D3D3D3')
#             venn_diagram.get_patch_by_id('001').set_alpha(0.7)
#         elif patch == '110':  # HIQ and LIQ overlap
#             venn_diagram.get_patch_by_id('110').set_color('#8B7D9B')
#             venn_diagram.get_patch_by_id('110').set_alpha(0.7)
#         elif patch == '101':  # HIQ and No IQ overlap
#             venn_diagram.get_patch_by_id('101').set_color('#B8C5D1')
#             venn_diagram.get_patch_by_id('101').set_alpha(0.7)
#         elif patch == '011':  # LIQ and No IQ overlap
#             venn_diagram.get_patch_by_id('011').set_color('#6B8FA3')
#             venn_diagram.get_patch_by_id('011').set_alpha(0.7)
#         elif patch == '111':  # All three overlap
#             venn_diagram.get_patch_by_id('111').set_color('#7A8B9B')
#             venn_diagram.get_patch_by_id('111').set_alpha(0.7)

# plt.title('Mutation Distribution by IQ Group', fontsize=16, fontweight='bold', pad=20)
# plt.tight_layout()
# plt.show()

# Print summary statistics
print(f"\nSummary:")
print(f"High IQ only: {len(HIQ_set - LIQ_set - no_iq_set)}")
print(f"Low IQ only: {len(LIQ_set - HIQ_set - no_iq_set)}")
print(f"No IQ data only: {len(no_iq_set - HIQ_set - LIQ_set)}")
print(f"High IQ & Low IQ overlap: {len(HIQ_set & LIQ_set)}")
print(f"High IQ & No IQ overlap: {len(HIQ_set & no_iq_set)}")
print(f"Low IQ & No IQ overlap: {len(LIQ_set & no_iq_set)}")
print(f"All three groups: {len(HIQ_set & LIQ_set & no_iq_set)}")


In [ ]:
# Helper function to classify mutations as LGD or Dmis
def classify_mutation_type(row):
    """Classify a mutation as LGD or Dmis"""
    GeneEff = row["GeneEff"].split(";")[0]
    if GeneEff in ["frameshift", "splice_acceptor", "splice_donor", "start_lost", "stop_gained", "stop_lost"]:
        return "LGD"
    elif GeneEff == "missense":
        revel = row["REVEL"].split(";")[0]
        if revel != ".":
            try:
                if float(revel) > 0.5:
                    return "Dmis"
            except:
                pass
    return None

# Classify mutations by type for each IQ group
HIQ_LGD = HIQ_Muts[HIQ_Muts.apply(classify_mutation_type, axis=1) == "LGD"]
HIQ_Dmis = HIQ_Muts[HIQ_Muts.apply(classify_mutation_type, axis=1) == "Dmis"]

LIQ_LGD = LIQ_Muts[LIQ_Muts.apply(classify_mutation_type, axis=1) == "LGD"]
LIQ_Dmis = LIQ_Muts[LIQ_Muts.apply(classify_mutation_type, axis=1) == "Dmis"]

if "IID" in HighConfMuts.columns:
    muts_without_iq = HighConfMuts[~HighConfMuts["IID"].isin(HIQ_IIDs.union(LIQ_IIDs))]
    NoIQ_LGD = muts_without_iq[muts_without_iq.apply(classify_mutation_type, axis=1) == "LGD"]
    NoIQ_Dmis = muts_without_iq[muts_without_iq.apply(classify_mutation_type, axis=1) == "Dmis"]
else:
    NoIQ_LGD = pd.DataFrame()
    NoIQ_Dmis = pd.DataFrame()

# Count mutations by type and IQ group
n_hiq_lgd = len(HIQ_LGD)
n_hiq_dmis = len(HIQ_Dmis)
n_liq_lgd = len(LIQ_LGD)
n_liq_dmis = len(LIQ_Dmis)
n_noiq_lgd = len(NoIQ_LGD)
n_noiq_dmis = len(NoIQ_Dmis)

# Create Stacked Bar Plot with All, LGD, and Dmis bars
fig, ax = plt.subplots(figsize=(12, 6), dpi=150)

# Prepare data
categories = ['All', 'LGD', 'Dmis']
x = np.arange(len(categories))
width = 0.6

# Stack data: [High IQ, Low IQ, No IQ] for each mutation type
all_counts = [n_hiq, n_liq, n_no_iq]  # All mutations combined
lgd_counts = [n_hiq_lgd, n_liq_lgd, n_noiq_lgd]
dmis_counts = [n_hiq_dmis, n_liq_dmis, n_noiq_dmis]

# Create stacked bars for All mutations
bars_all_hiq = ax.bar(x[0], all_counts[0], width, label='High IQ (IQ > 70)', 
                       color='#AED6F1', edgecolor='black', linewidth=1.5)
bars_all_liq = ax.bar(x[0], all_counts[1], width, bottom=all_counts[0],
                       label='Low IQ (IQ ≤ 70)', 
                       color='#21618C', edgecolor='black', linewidth=1.5)
bars_all_noiq = ax.bar(x[0], all_counts[2], width, bottom=np.array(all_counts[0]) + np.array(all_counts[1]),
                        label='No IQ Data', 
                        color='#D3D3D3', edgecolor='black', linewidth=1.5)

# Create stacked bars for LGD
bars_lgd_hiq = ax.bar(x[1], lgd_counts[0], width, 
                      color='#AED6F1', edgecolor='black', linewidth=1.5)
bars_lgd_liq = ax.bar(x[1], lgd_counts[1], width, bottom=lgd_counts[0],
                      color='#21618C', edgecolor='black', linewidth=1.5)
bars_lgd_noiq = ax.bar(x[1], lgd_counts[2], width, bottom=np.array(lgd_counts[0]) + np.array(lgd_counts[1]),
                       color='#D3D3D3', edgecolor='black', linewidth=1.5)

# Create stacked bars for Dmis
bars_dmis_hiq = ax.bar(x[2], dmis_counts[0], width, 
                       color='#AED6F1', edgecolor='black', linewidth=1.5)
bars_dmis_liq = ax.bar(x[2], dmis_counts[1], width, bottom=dmis_counts[0],
                       color='#21618C', edgecolor='black', linewidth=1.5)
bars_dmis_noiq = ax.bar(x[2], dmis_counts[2], width, 
                        bottom=np.array(dmis_counts[0]) + np.array(dmis_counts[1]),
                        color='#D3D3D3', edgecolor='black', linewidth=1.5)

# Add value labels on bars
def add_value_label(bar_container, bottom_value=None):
    """Add a label to a single bar (bar_container is a BarContainer from ax.bar())"""
    # Access the first (and only) bar in the container
    bar = bar_container[0]
    height = bar.get_height()
    if height > 0:  # Only label if height > 0
        if bottom_value is not None:
            y_pos = bottom_value + height/2
        else:
            y_pos = height/2
        ax.text(bar.get_x() + bar.get_width()/2., y_pos,
                f'{int(height)}',
                ha='center', va='center', fontweight='bold', fontsize=11)

# Add labels for All mutations bar
add_value_label(bars_all_hiq)
add_value_label(bars_all_liq, all_counts[0])
add_value_label(bars_all_noiq, all_counts[0] + all_counts[1])

# Add labels for LGD bar
add_value_label(bars_lgd_hiq)
add_value_label(bars_lgd_liq, lgd_counts[0])
add_value_label(bars_lgd_noiq, lgd_counts[0] + lgd_counts[1])

# Add labels for Dmis bar
add_value_label(bars_dmis_hiq)
add_value_label(bars_dmis_liq, dmis_counts[0])
add_value_label(bars_dmis_noiq, dmis_counts[0] + dmis_counts[1])

# Customize plot
ax.set_ylabel('Number of Mutations', fontsize=14, fontweight='bold')
ax.set_title('Mutation Counts by Type and IQ Group', fontsize=16, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=12, fontweight='bold')

# Place legend to the right
ax.legend(
    loc='upper right',
    #bbox_to_anchor=(1.02, 0.5),
    fontsize=11, frameon=True, fancybox=True, shadow=True
)

ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)

# Set y-axis to start from 0
max_height = max(sum(all_counts), sum(lgd_counts), sum(dmis_counts))
ax.set_ylim(0, max_height * 1.15)

plt.tight_layout()
plt.show()

# Print detailed breakdown
print(f"\nDetailed breakdown by mutation type:")
print(f"\nAll mutations:")
print(f"  High IQ: {n_hiq} ({n_hiq/sum(all_counts)*100:.1f}%)" if sum(all_counts) > 0 else f"  High IQ: {n_hiq}")
print(f"  Low IQ: {n_liq} ({n_liq/sum(all_counts)*100:.1f}%)" if sum(all_counts) > 0 else f"  Low IQ: {n_liq}")
print(f"  No IQ Data: {n_no_iq} ({n_no_iq/sum(all_counts)*100:.1f}%)" if sum(all_counts) > 0 else f"  No IQ Data: {n_no_iq}")
print(f"  Total All: {sum(all_counts)}")

print(f"\nLGD mutations:")
print(f"  High IQ: {n_hiq_lgd} ({n_hiq_lgd/sum(lgd_counts)*100:.1f}%)" if sum(lgd_counts) > 0 else f"  High IQ: {n_hiq_lgd}")
print(f"  Low IQ: {n_liq_lgd} ({n_liq_lgd/sum(lgd_counts)*100:.1f}%)" if sum(lgd_counts) > 0 else f"  Low IQ: {n_liq_lgd}")
print(f"  No IQ Data: {n_noiq_lgd} ({n_noiq_lgd/sum(lgd_counts)*100:.1f}%)" if sum(lgd_counts) > 0 else f"  No IQ Data: {n_noiq_lgd}")
print(f"  Total LGD: {sum(lgd_counts)}")

print(f"\nDmis mutations:")
print(f"  High IQ: {n_hiq_dmis} ({n_hiq_dmis/sum(dmis_counts)*100:.1f}%)" if sum(dmis_counts) > 0 else f"  High IQ: {n_hiq_dmis}")
print(f"  Low IQ: {n_liq_dmis} ({n_liq_dmis/sum(dmis_counts)*100:.1f}%)" if sum(dmis_counts) > 0 else f"  Low IQ: {n_liq_dmis}")
print(f"  No IQ Data: {n_noiq_dmis} ({n_noiq_dmis/sum(dmis_counts)*100:.1f}%)" if sum(dmis_counts) > 0 else f"  No IQ Data: {n_noiq_dmis}")
print(f"  Total Dmis: {sum(dmis_counts)}")

print(f"\nOverall totals:")
print(f"  High IQ: {n_hiq_lgd + n_hiq_dmis} (LGD: {n_hiq_lgd}, Dmis: {n_hiq_dmis})")
print(f"  Low IQ: {n_liq_lgd + n_liq_dmis} (LGD: {n_liq_lgd}, Dmis: {n_liq_dmis})")
print(f"  No IQ Data: {n_noiq_lgd + n_noiq_dmis} (LGD: {n_noiq_lgd}, Dmis: {n_noiq_dmis})")


In [ ]:
# Alternative: Grouped Bar Plot (non-stacked) for better comparison
fig, ax = plt.subplots(figsize=(10, 6), dpi=150)

categories = ['High IQ\n(IQ > 70)', 'Low IQ\n(IQ ≤ 70)', 'No IQ Data']
counts = [n_hiq, n_liq, n_no_iq]
colors = ['#AED6F1', '#21618C', '#D3D3D3']

bars = ax.bar(categories, counts, color=colors, edgecolor='black', linewidth=1.5, width=0.6)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontweight='bold', fontsize=12)

# Customize plot
ax.set_ylabel('Number of Mutations', fontsize=14, fontweight='bold')
ax.set_title('Mutation Counts by IQ Group', fontsize=16, fontweight='bold', pad=15)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)
ax.set_ylim(0, max(counts) * 1.2)

plt.tight_layout()
plt.show()


In [ ]:
# Additional analysis: Gene overlap between HIQ and LIQ
if "HGNC" in HIQ_Muts.columns and "HGNC" in LIQ_Muts.columns:
    HIQ_genes = set(HIQ_Muts["HGNC"].unique())
    LIQ_genes = set(LIQ_Muts["HGNC"].unique())
    
    genes_hiq_only = HIQ_genes - LIQ_genes
    genes_liq_only = LIQ_genes - HIQ_genes
    genes_overlap = HIQ_genes & LIQ_genes
    
    print(f"\nGene-level analysis:")
    print(f"Total unique genes in High IQ: {len(HIQ_genes)}")
    print(f"Total unique genes in Low IQ: {len(LIQ_genes)}")
    print(f"Genes in High IQ only: {len(genes_hiq_only)}")
    print(f"Genes in Low IQ only: {len(genes_liq_only)}")
    print(f"Genes overlapping between High and Low IQ: {len(genes_overlap)}")
    
    # Create gene overlap visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), dpi=150)
    
    # Gene Venn diagram
    venn_genes = venn.venn2(
        [HIQ_genes, LIQ_genes],
        set_labels=('High IQ Genes', 'Low IQ Genes'),
        ax=ax1
    )
    
    if venn_genes:
        venn_genes.get_patch_by_id('10').set_color('#AED6F1')
        venn_genes.get_patch_by_id('10').set_alpha(0.7)
        venn_genes.get_patch_by_id('01').set_color('#21618C')
        venn_genes.get_patch_by_id('01').set_alpha(0.7)
        venn_genes.get_patch_by_id('11').set_color('#8B7D9B')
        venn_genes.get_patch_by_id('11').set_alpha(0.7)
    
    ax1.set_title('Gene Overlap Between IQ Groups', fontsize=14, fontweight='bold')
    
    # Gene count bar plot
    gene_categories = ['High IQ\nOnly', 'Low IQ\nOnly', 'Overlap']
    gene_counts = [len(genes_hiq_only), len(genes_liq_only), len(genes_overlap)]
    gene_colors = ['#AED6F1', '#21618C', '#8B7D9B']
    
    bars = ax2.bar(gene_categories, gene_counts, color=gene_colors, 
                   edgecolor='black', linewidth=1.5, width=0.6)
    
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontweight='bold', fontsize=12)
    
    ax2.set_ylabel('Number of Genes', fontsize=12, fontweight='bold')
    ax2.set_title('Gene Counts by IQ Group', fontsize=14, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    ax2.set_axisbelow(True)
    ax2.set_ylim(0, max(gene_counts) * 1.2)
    
    plt.tight_layout()
    plt.show()


In [ ]:
HighConfMuts
HIQ_Muts
LIQ_Muts

###### IQ Distribution

In [ ]:
ASC_IQ_dat["IQ"].hist(label="ASC", density=1, alpha=0.5)
print(ASC_IQ_dat["IQ"].mean())

Spark_IQ_dat["fsiq"].hist(label="Spark", density=1, alpha=0.5)
print(Spark_IQ_dat["fsiq"].mean())

plt.legend()

In [ ]:
# Create detailed stacked bar plot for each gene showing HIQ, LIQ, and No IQ mutations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Get all unique genes from HighConfMuts
all_genes = sorted(HighConfMuts["HGNC"].unique())

# Get IIDs with and without IQ data
all_iids_with_iq = set(HighIQ).union(set(LowIQ))
muts_without_iq = HighConfMuts[~HighConfMuts["IID"].isin(all_iids_with_iq)]

# Count mutations per gene for each category
gene_counts = []
for gene in all_genes:
    hiq_count = len(HIQ_Muts[HIQ_Muts["HGNC"] == gene])
    liq_count = len(LIQ_Muts[LIQ_Muts["HGNC"] == gene])
    no_iq_count = len(muts_without_iq[muts_without_iq["HGNC"] == gene])
    gene_counts.append({
        'gene': gene,
        'HIQ': hiq_count,
        'LIQ': liq_count,
        'No_IQ': no_iq_count,
        'total': hiq_count + liq_count + no_iq_count
    })

# Convert to DataFrame and sort by total mutations (descending)
gene_counts_df = pd.DataFrame(gene_counts)
gene_counts_df = gene_counts_df.sort_values('total', ascending=False)

# Create the stacked bar plot
fig, ax = plt.subplots(figsize=(max(12, len(all_genes) * 0.5), 12), dpi=240)

x = np.arange(len(gene_counts_df))
width = 0.8

# Stack bars: No IQ (bottom), LIQ (middle), HIQ (top)
# Bottom layer: No IQ data
bars_no_iq = ax.bar(x, gene_counts_df['No_IQ'].values, width, 
                     label='No IQ Data', color='#D3D3D3', edgecolor='black', linewidth=1)

# Middle layer: Low IQ (stacked on top of No IQ)
bars_liq = ax.bar(x, gene_counts_df['LIQ'].values, width,
                   bottom=gene_counts_df['No_IQ'].values,
                   label='Low IQ (IQ ≤ 70)', color='#21618C', edgecolor='black', linewidth=1)

# Top layer: High IQ (stacked on top of LIQ + No IQ)
bars_hiq = ax.bar(x, gene_counts_df['HIQ'].values, width,
                   bottom=gene_counts_df['No_IQ'].values + gene_counts_df['LIQ'].values,
                   label='High IQ (IQ > 70)', color='#AED6F1', edgecolor='black', linewidth=1)

# Add value labels on bars (only if count > 0)
def add_labels(bars, values, bottom_values=None):
    for i, (bar, val) in enumerate(zip(bars, values)):
        if val > 0:
            if bottom_values is not None:
                y_pos = bottom_values[i] + val / 2
            else:
                y_pos = val / 2
            ax.text(bar.get_x() + bar.get_width()/2., y_pos,
                    f'{int(val)}',
                    ha='center', va='center', fontweight='bold', fontsize=16)

# Add labels for each segment
add_labels(bars_no_iq, gene_counts_df['No_IQ'].values)
add_labels(bars_liq, gene_counts_df['LIQ'].values, gene_counts_df['No_IQ'].values)
add_labels(bars_hiq, gene_counts_df['HIQ'].values, 
           gene_counts_df['No_IQ'].values + gene_counts_df['LIQ'].values)

# Customize plot
ax.set_xlabel('Gene', fontsize=28, fontweight='bold')
ax.set_ylabel('Number of Mutations', fontsize=28, fontweight='bold')
ax.set_title('Mutation Counts by Gene and IQ Group', fontsize=32, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels(gene_counts_df['gene'].values, rotation=45, ha='right', fontsize=20)

# Add legend
ax.legend(loc='upper right', fontsize=22, frameon=True, fancybox=True, shadow=True)

# Add grid
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)

# Set y-axis tick label font size
ax.tick_params(axis='y', labelsize=20)

# Set y-axis to start from 0
max_height = gene_counts_df['total'].max()
ax.set_ylim(0, max_height * 1.15)

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nSummary by gene:")
print(f"Total genes: {len(all_genes)}")
print(f"Genes with mutations: {len(gene_counts_df[gene_counts_df['total'] > 0])}")
print(f"\nTop 10 genes by total mutations:")
print(gene_counts_df.head(10)[['gene', 'HIQ', 'LIQ', 'No_IQ', 'total']].to_string(index=False))


In [ ]:
#HIQ_GW = Mut2GeneDF(HIQ_Muts, PPVs=HIQ_Rate) # Not use weights input
HIQ_GW = Mut2GeneDF(HIQ_Muts, LGD=True, Dmis=True) 
HIQ_GW_LGD = Mut2GeneDF(HIQ_Muts, LGD=True, Dmis=False) 
HIQ_GW_Dmis = Mut2GeneDF(HIQ_Muts, LGD=False, Dmis=True) 

Dict2Fil(HIQ_GW, "/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/ASD.HIQ.gw.csv")
HIQ_DF = MouseSTR_AvgZ_Weighted(ExpZ2Mat, HIQ_GW, csv_fil = "../dat/Unionize_bias/ASD.HIQ_spec.bias.csv")

#LIQ_GW = Mut2GeneDF(LIQ_Muts, PPVs=LIQ_Rate)
LIQ_GW = Mut2GeneDF(LIQ_Muts, LGD=True, Dmis=True)
Dict2Fil(LIQ_GW, "/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/ASD.LIQ.gw.csv")
LIQ_DF = MouseSTR_AvgZ_Weighted(ExpZ2Mat, LIQ_GW, csv_fil = "../dat/Unionize_bias/ASD.LIQ_spec.bias.csv")

##HIQ_DF = HIQ_DF.set_index("STR")
#HIQ_LGD = HIQ_LGD.set_index("STR")
#HIQ_Dmis = HIQ_Dmis.set_index("STR")
#LIQ_DF = LIQ_DF.set_index("STR")
#LIQ_LGD = LIQ_LGD.set_index("STR")
#LIQ_Dmis = LIQ_Dmis.set_index("STR")

In [ ]:
ASD_CircuitsSet = pd.read_csv(
    "/home/jw3514/Work/ASD_Circuits/notebooks/ASD.SA.Circuits.Size46.csv",
    index_col="idx")
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")
#ASD_Circuits.append("Field_CA3")

#CIR_REGIONS = ['Isocortex', 'Hippocampus', 
#     'Cortical_subplate', 'Amygdalar', 'Striatum', 'Thalamus', 'Olfactory_areas']
CIR_REGIONS = ['Isocortex', 'Hippocampus', 
     'Amygdala', 'Striatum', 'Thalamus', 'Olfactory_areas']
#CIR_REGIONS = ['Isocortex', 'Hippocampus', 
#     'Amygdala', 'Striatum', 'Thalamus']

CIR_REGIONS_Dict = {}
for i in range(len(CIR_REGIONS)):
    CIR_REGIONS_Dict[CIR_REGIONS[i]] = []
for _str in ASD_Circuits:
    for i in range(len(CIR_REGIONS)):
        if str2reg[_str] == CIR_REGIONS[i]:
            CIR_REGIONS_Dict[CIR_REGIONS[i]].append(_str)
            break
CIR_REGIONS_Dict["Amygdala"].append("Bed_nuclei_of_the_stria_terminalis")

In [ ]:
ASD_CircuitsSet

---

## Subsampling Analysis: Testing if LIQ Pattern Holds After Matching Sample Size


In [ ]:
# Subsample LIQ mutations to match HIQ size and test if pattern holds
# Parameters
N_ITERATIONS = 100  # Number of subsampling iterations
RANDOM_SEED = 42  # For reproducibility

# Get sizes
n_hiq = len(HIQ_Muts)
n_liq = len(LIQ_Muts)
print(f"High IQ mutations: {n_hiq}")
print(f"Low IQ mutations: {n_liq}")
print(f"Subsampling LIQ to {n_hiq} mutations, {N_ITERATIONS} times...")

# Calculate HIQ structural bias once (baseline)
HIQ_GW = Mut2GeneDF(HIQ_Muts, LGD=True, Dmis=True)
HIQ_DF = MouseSTR_AvgZ_Weighted(ExpZ2Mat, HIQ_GW, csv_fil=None)  # Don't save intermediate files
HIQ_DF = HIQ_DF.set_index("STR") if "STR" in HIQ_DF.columns else HIQ_DF

# Get ASD circuits for analysis
ASD_CircuitsSet = pd.read_csv(
    "/home/jw3514/Work/ASD_Circuits/notebooks/ASD.SA.Circuits.Size46.csv",
    index_col="idx")
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")

CIR_REGIONS = ['Isocortex', 'Hippocampus', 'Amygdala', 'Striatum', 'Thalamus', 'Olfactory_areas']
CIR_REGIONS_Dict = {}
for i in range(len(CIR_REGIONS)):
    CIR_REGIONS_Dict[CIR_REGIONS[i]] = []
for _str in ASD_Circuits:
    for i in range(len(CIR_REGIONS)):
        if str2reg[_str] == CIR_REGIONS[i]:
            CIR_REGIONS_Dict[CIR_REGIONS[i]].append(_str)
            break
CIR_REGIONS_Dict["Amygdala"].append("Bed_nuclei_of_the_stria_terminalis")

HIQ_Cir = HIQ_DF.loc[ASD_Circuits, :]
HIQ_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdala"

# Calculate HIQ regional biases
HIQ_regional_biases = {}
for REG in CIR_REGIONS:
    HIQ_regional_biases[REG] = HIQ_Cir[HIQ_Cir["REGION"] == REG]["EFFECT"].mean()

print(f"\nHIQ regional biases:")
for REG, bias in HIQ_regional_biases.items():
    print(f"  {REG}: {bias:.4f}")

# Store results for each subsample (region level)
subsample_results = {
    'iteration': [],
    'region': [],
    'bias': []
}

# Store structure-level results for each subsample
subsample_str_results = {
    'iteration': [],
    'str': [],
    'region': [],
    'bias': []
}

# Perform subsampling
np.random.seed(RANDOM_SEED)
for iteration in range(N_ITERATIONS):
    # Randomly subsample LIQ mutations
    LIQ_subsampled = LIQ_Muts.sample(n=n_hiq, replace=False, random_state=RANDOM_SEED + iteration)
    
    # Calculate structural bias for subsampled LIQ
    LIQ_GW = Mut2GeneDF(LIQ_subsampled, LGD=True, Dmis=True)
    LIQ_DF = MouseSTR_AvgZ_Weighted(ExpZ2Mat, LIQ_GW, csv_fil=None)
    LIQ_DF = LIQ_DF.set_index("STR") if "STR" in LIQ_DF.columns else LIQ_DF
    
    LIQ_Cir = LIQ_DF.loc[ASD_Circuits, :]
    LIQ_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdala"
    
    # Calculate regional biases
    for REG in CIR_REGIONS:
        bias = LIQ_Cir[LIQ_Cir["REGION"] == REG]["EFFECT"].mean()
        subsample_results['iteration'].append(iteration)
        subsample_results['region'].append(REG)
        subsample_results['bias'].append(bias)
    
    # Store structure-level biases
    for str_name in LIQ_Cir.index:
        str_bias = LIQ_Cir.loc[str_name, "EFFECT"]
        str_region = LIQ_Cir.loc[str_name, "REGION"]
        subsample_str_results['iteration'].append(iteration)
        subsample_str_results['str'].append(str_name)
        subsample_str_results['region'].append(str_region)
        subsample_str_results['bias'].append(str_bias)
    
    if (iteration + 1) % 10 == 0:
        print(f"Completed {iteration + 1}/{N_ITERATIONS} iterations...")

# Convert to DataFrame for analysis
subsample_df = pd.DataFrame(subsample_results)
subsample_str_df = pd.DataFrame(subsample_str_results)

# Store HIQ structure-level biases for comparison
HIQ_str_biases = {}
for str_name in HIQ_Cir.index:
    HIQ_str_biases[str_name] = HIQ_Cir.loc[str_name, "EFFECT"]

print(f"\nSubsampling complete!")
print(f"Total subsamples analyzed: {N_ITERATIONS}")
print(f"Structure-level data stored for {len(HIQ_Cir)} structures")


In [ ]:
# Analyze subsampling results and compare to HIQ
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate summary statistics for subsampled LIQ
liq_subsample_summary = subsample_df.groupby('region')['bias'].agg(['mean', 'std', 'min', 'max']).reset_index()
liq_subsample_summary.columns = ['region', 'mean_bias', 'std_bias', 'min_bias', 'max_bias']

# Add HIQ biases for comparison
hiq_comparison = pd.DataFrame({
    'region': list(HIQ_regional_biases.keys()),
    'HIQ_bias': list(HIQ_regional_biases.values())
})
comparison_df = liq_subsample_summary.merge(hiq_comparison, on='region')

# Calculate difference (LIQ - HIQ)
comparison_df['bias_diff'] = comparison_df['mean_bias'] - comparison_df['HIQ_bias']

print("\n" + "="*80)
print("COMPARISON: Subsampled LIQ vs HIQ")
print("="*80)
print(comparison_df.to_string(index=False))

# Test if LIQ bias is consistently higher/lower than HIQ across regions
# Count how many iterations show LIQ > HIQ for each region
liq_higher_count = {}
for REG in CIR_REGIONS:
    liq_biases = subsample_df[subsample_df['region'] == REG]['bias'].values
    hiq_bias = HIQ_regional_biases[REG]
    liq_higher_count[REG] = np.sum(liq_biases > hiq_bias)

print(f"\n" + "="*80)
print("CONSISTENCY CHECK: How many subsamples show LIQ > HIQ?")
print("="*80)
for REG in CIR_REGIONS:
    count = liq_higher_count[REG]
    pct = (count / N_ITERATIONS) * 100
    print(f"{REG:20s}: {count:3d}/{N_ITERATIONS} ({pct:5.1f}%)")

# Calculate consistency: variance across regions (lower variance = more consistent)
hiq_variance = np.var(list(HIQ_regional_biases.values()))
liq_variances = []
for iteration in range(N_ITERATIONS):
    iter_biases = subsample_df[subsample_df['iteration'] == iteration]['bias'].values
    liq_variances.append(np.var(iter_biases))

liq_mean_variance = np.mean(liq_variances)
liq_std_variance = np.std(liq_variances)

print(f"\n" + "="*80)
print("CONSISTENCY ACROSS REGIONS (variance of regional biases)")
print("="*80)
print(f"HIQ variance: {hiq_variance:.6f}")
print(f"LIQ subsampled mean variance: {liq_mean_variance:.6f} ± {liq_std_variance:.6f}")
print(f"LIQ subsampled variance range: [{np.min(liq_variances):.6f}, {np.max(liq_variances):.6f}]")


In [ ]:
# Visualize subsampling results
fig, axes = plt.subplots(2, 2, figsize=(16, 12), dpi=150)

# Panel 1: Regional biases comparison (boxplot)
ax1 = axes[0, 0]
region_order = CIR_REGIONS
positions = np.arange(len(region_order))

# Prepare data for boxplot
boxplot_data = []
boxplot_labels = []
for REG in region_order:
    liq_biases = subsample_df[subsample_df['region'] == REG]['bias'].values
    boxplot_data.append(liq_biases)
    boxplot_labels.append(REG)

bp = ax1.boxplot(boxplot_data, positions=positions, widths=0.6, patch_artist=True,
                 labels=boxplot_labels, showmeans=True)

# Color boxes
colors = ['#21618C' if i % 2 == 0 else '#5DADE2' for i in range(len(bp['boxes']))]
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

# Add HIQ points
for i, REG in enumerate(region_order):
    hiq_bias = HIQ_regional_biases[REG]
    ax1.scatter(i+1, hiq_bias, color='#AED6F1', s=150, marker='D', 
               edgecolor='black', linewidth=2, zorder=10, label='HIQ' if i == 0 else '')

ax1.set_ylabel('Structural Bias', fontsize=14, fontweight='bold')
ax1.set_title('Regional Bias: Subsampled LIQ (boxplots) vs HIQ (diamonds)', 
             fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
ax1.legend(fontsize=11)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Panel 2: Distribution of bias differences (LIQ - HIQ) by region
ax2 = axes[0, 1]
for REG in region_order:
    liq_biases = subsample_df[subsample_df['region'] == REG]['bias'].values
    hiq_bias = HIQ_regional_biases[REG]
    diffs = liq_biases - hiq_bias
    ax2.hist(diffs, bins=20, alpha=0.6, label=REG, density=True)

ax2.axvline(x=0, color='black', linestyle='--', linewidth=2, label='No difference')
ax2.set_xlabel('Bias Difference (LIQ - HIQ)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Density', fontsize=12, fontweight='bold')
ax2.set_title('Distribution of Bias Differences by Region', fontsize=14, fontweight='bold')
ax2.legend(fontsize=9, loc='upper right')
ax2.grid(alpha=0.3)

# Panel 3: Consistency (variance) comparison
ax3 = axes[1, 0]
ax3.hist(liq_variances, bins=30, alpha=0.7, color='#21618C', edgecolor='black', label='LIQ subsampled')
ax3.axvline(x=hiq_variance, color='#AED6F1', linestyle='--', linewidth=3, label=f'HIQ (variance={hiq_variance:.6f})')
ax3.axvline(x=liq_mean_variance, color='red', linestyle='--', linewidth=2, 
           label=f'LIQ mean (variance={liq_mean_variance:.6f})')
ax3.set_xlabel('Variance of Regional Biases', fontsize=12, fontweight='bold')
ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax3.set_title('Consistency Across Regions (Lower Variance = More Consistent)', 
             fontsize=14, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(alpha=0.3)

# Panel 4: Percentage of iterations where LIQ > HIQ by region
ax4 = axes[1, 1]
region_names = list(liq_higher_count.keys())
percentages = [(liq_higher_count[REG] / N_ITERATIONS) * 100 for REG in region_names]
colors_bar = ['#21618C' if p > 50 else '#AED6F1' for p in percentages]

bars = ax4.barh(region_names, percentages, color=colors_bar, edgecolor='black', linewidth=1.5)
ax4.axvline(x=50, color='black', linestyle='--', linewidth=2, label='50% (random)')
ax4.set_xlabel('% of Subsamples with LIQ > HIQ', fontsize=12, fontweight='bold')
ax4.set_title('Consistency: LIQ Higher than HIQ', fontsize=14, fontweight='bold')
ax4.set_xlim(0, 100)
ax4.legend(fontsize=11)
ax4.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (bar, pct) in enumerate(zip(bars, percentages)):
    width = bar.get_width()
    ax4.text(width + 1, bar.get_y() + bar.get_height()/2, 
            f'{pct:.1f}%', ha='left', va='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n" + "="*80)
print("SUMMARY: Does the pattern hold after subsampling?")
print("="*80)
print(f"\n1. Consistency across regions:")
print(f"   - HIQ variance: {hiq_variance:.6f}")
print(f"   - LIQ mean variance: {liq_mean_variance:.6f}")
if liq_mean_variance < hiq_variance:
    print(f"   → LIQ shows MORE consistency (lower variance) than HIQ")
else:
    print(f"   → LIQ shows LESS consistency (higher variance) than HIQ")

print(f"\n2. Direction of bias:")
regions_liq_higher = [REG for REG in region_order if liq_higher_count[REG] > 50]
regions_hiq_higher = [REG for REG in region_order if liq_higher_count[REG] < 50]
print(f"   - Regions where LIQ > HIQ in >50% subsamples: {len(regions_liq_higher)}/{len(region_order)}")
print(f"   - Regions where HIQ > LIQ in >50% subsamples: {len(regions_hiq_higher)}/{len(region_order)}")

if len(regions_liq_higher) > len(regions_hiq_higher):
    print(f"   → Pattern suggests LIQ has higher bias across most regions")
elif len(regions_liq_higher) < len(regions_hiq_higher):
    print(f"   → Pattern suggests HIQ has higher bias across most regions")
else:
    print(f"   → Mixed pattern across regions")


In [ ]:
# Structure-level plotting: Similar to main figure
from matplotlib.markers import TICKDOWN

def drop_fromList(_list):
    return [" ".join(x.split("_")) for x in _list]

def significance_bar(start, end, height, displaystring, linewidth=1.2,
                     markersize=8, boxpad=0.3, fontsize=15, color='k'):
    # draw a line with downticks at the ends
    plt.plot([start, end], [height]*2, '-', color=color, lw=linewidth, marker=TICKDOWN,
             markeredgewidth=linewidth, markersize=markersize)
    # draw the text with a bounding box covering up the line
    plt.text(0.5*(start+end), height + 0.03, displaystring, ha='center', va='center', fontsize=fontsize)

# Calculate mean and std for each structure across subsamples
str_summary = subsample_str_df.groupby('str')['bias'].agg(['mean', 'std']).reset_index()
str_summary.columns = ['str', 'mean_bias', 'std_bias']

# Add HIQ biases
str_summary['HIQ_bias'] = str_summary['str'].map(HIQ_str_biases)

# Add region information
str_regions = {}
for str_name in HIQ_Cir.index:
    str_regions[str_name] = HIQ_Cir.loc[str_name, "REGION"]
str_summary['region'] = str_summary['str'].map(str_regions)

# Create plots for each region
fig = plt.figure(figsize=(30, 12), dpi=150)
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])

# Define colors for each region (matching main figure)
region_colors = {
    'Isocortex': '#268ad5',
    'Olfactory_areas': '#5ab4ac',
    'Cortical_subplate': '#7ac3fa',
    'Hippocampus': '#2c9d39',
    'Amygdala': '#742eb5',
    'Striatum': '#ed8921',
    'Thalamus': '#e82315',
    'Midbrain': '#f6b26b',
    'Pallidum': '#2ECC71',
}

x_offset = 0
all_ticks = []
all_positions = []

for REG in CIR_REGIONS:
    # Get structures in this region
    reg_strs = str_summary[str_summary['region'] == REG].copy()
    if len(reg_strs) == 0:
        continue
    
    # Sort by HIQ bias
    reg_strs = reg_strs.sort_values('HIQ_bias')
    
    X = np.arange(len(reg_strs))
    
    # Plot HIQ (diamonds)
    hiq_biases = reg_strs['HIQ_bias'].values
    ax.scatter(X + x_offset, hiq_biases, color='#AED6F1', s=200, marker='D',
              edgecolor='black', linewidth=2, zorder=10, label='HIQ' if REG == CIR_REGIONS[0] else '')
    
    # Plot LIQ subsampled (boxplots or error bars)
    for i, (idx, row) in enumerate(reg_strs.iterrows()):
        str_name = row['str']
        mean_bias = row['mean_bias']
        std_bias = row['std_bias']
        
        # Get all biases for this structure across iterations
        str_biases = subsample_str_df[subsample_str_df['str'] == str_name]['bias'].values
        
        # Plot error bar
        ax.errorbar(i + x_offset, mean_bias, yerr=std_bias,
                   fmt='o', markersize=8, color=region_colors.get(REG, '#21618C'),
                   markeredgecolor='black', markeredgewidth=1.5,
                   ecolor=region_colors.get(REG, '#21618C'), capsize=4, capthick=1.5,
                   alpha=0.8, zorder=5)
    
    # Store positions and labels
    all_positions.extend(X + x_offset)
    all_ticks.extend([drop_fromList([s])[0] for s in reg_strs['str'].values])
    
    x_offset += len(reg_strs)

# Set ticks and labels
ax.set_xticks(all_positions)
ax.set_xticklabels(all_ticks, rotation=45, ha='right', rotation_mode="anchor", fontsize=12)
ax.set_ylabel('Structural Bias', fontsize=20, fontweight='bold')
ax.set_xlabel('', fontsize=14)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)
ax.set_ylim(-0.2, 0.84)
ax.tick_params(axis='y', labelsize=16)

# Create legend
legend_ax1 = fig.add_axes([0.85, 0.7, 0.1, 0.2])
legend_ax1.axis('off')
hiq_marker = plt.Line2D([0], [0], marker='D', color='none', markerfacecolor='#AED6F1',
                        markeredgecolor='black', markersize=12, markeredgewidth=2, linestyle='None')
liq_marker = plt.Line2D([0], [0], marker='o', color='none', markerfacecolor='#21618C',
                       markeredgecolor='black', markersize=8, markeredgewidth=1.5, linestyle='None')
legend_ax1.legend([hiq_marker, liq_marker], ['HIQ', 'LIQ (subsampled, mean±SD)'], 
                 fontsize=14, frameon=False)

# Create region color legend
legend_ax2 = fig.add_axes([0.85, 0.4, 0.1, 0.25])
legend_ax2.axis('off')
region_handles = [plt.Rectangle((0, 0), 1, 1, facecolor=region_colors.get(REG, '#21618C'), 
                                edgecolor='black') for REG in CIR_REGIONS if REG in region_colors]
legend_ax2.legend(region_handles, [REG for REG in CIR_REGIONS if REG in region_colors], 
                 fontsize=12, frameon=False)

plt.title('Structure-Level Bias: HIQ vs Subsampled LIQ (100 iterations)', 
         fontsize=18, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Print structure-level summary statistics
print("\n" + "="*80)
print("STRUCTURE-LEVEL SUMMARY")
print("="*80)
print(f"\nTotal structures analyzed: {len(str_summary)}")
print(f"\nStructures where LIQ mean > HIQ:")
liq_higher_strs = str_summary[str_summary['mean_bias'] > str_summary['HIQ_bias']]
print(f"  Count: {len(liq_higher_strs)}/{len(str_summary)} ({len(liq_higher_strs)/len(str_summary)*100:.1f}%)")
print(f"\nStructures where LIQ mean < HIQ:")
liq_lower_strs = str_summary[str_summary['mean_bias'] < str_summary['HIQ_bias']]
print(f"  Count: {len(liq_lower_strs)}/{len(str_summary)} ({len(liq_lower_strs)/len(str_summary)*100:.1f}%)")


In [ ]:
print(RegionDistributionsList(ASD_Circuits))

In [ ]:
HIQ_Cir = HIQ_DF.loc[ASD_Circuits,:]
LIQ_Cir = LIQ_DF.loc[ASD_Circuits,:]
HIQ_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdala"
LIQ_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdala"

### Individual STRs

In [ ]:
IQ_PhenotypeDF = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/IQ_Bias_Diff.Pvalues.csv", index_col="STR")

In [ ]:
IQ_PhenotypeDF.head(2)

In [ ]:
import re
from matplotlib.markers import TICKDOWN

def drop_fromList(_list):
    return [" ".join(x.split("_")) for x in _list]
    
def PlotPhenotype(Dat1, Err1, Dat2, Err2, REG, IQ_PhenotypeDF, label1="", label2 = "", fig_size=0):
    #sort_idx = np.argsort(HighIQ_dat)
    Pvalues = IQ_PhenotypeDF.loc[Dat1.index.values, "Pvalue"].values
    sort_idx = np.argsort(Dat1["EFFECT"].values)
    X = np.arange(Dat1.shape[0])
    if fig_size == 0:
        fig_size = (2.5 + 0.4 * Dat1.shape[0], 6)
    print(fig_size)
    fig, ax = plt.subplots(dpi=120, figsize = fig_size)
    
    dat1 = Dat1["EFFECT"].values[sort_idx]
    err1 = Err1[sort_idx]
    dat2 = Dat2["EFFECT"].values[sort_idx]
    err2 = Err2[sort_idx]
    
    bar_width = 0.3
    ax.bar(X - 0.15, dat1, yerr = err1, color="#AED6F1", width=0.3, label=label1, 
           edgecolor ='black')
    ax.bar(X + 0.15, dat2, yerr = err2, color="#21618C", width=0.3, label=label2, 
       edgecolor ='black', hatch="")
    
    Pvalues = Pvalues[sort_idx]
    offset  = 0.05
    for i,p in enumerate(Pvalues):
        if p>=0.05:
            continue
        elif p<0.001:
            displaystring = r'***'
        elif p<0.01:
            displaystring = r'**'
        else:
            displaystring = r'*'

        height = offset +  max(dat1[i] + err1[i], dat2[i]+err2[i])
        bar_centers = X[i]-0.3 + np.array([0.5,1.5])*bar_width
        significance_bar(bar_centers[0],bar_centers[1],height,displaystring)
    
    ax.set_xlabel('', fontweight ='bold', fontsize = 15)
    ax.set_ylabel('Expression Bias', fontsize = 15)
    plt.xticks(X, drop_fromList(Dat1.index.values[sort_idx]), rotation=45, fontsize=10, ha='right', rotation_mode="anchor")

    #ax.legend(loc="best", fontsize=15)
    plt.title(" ".join(REG.split("_")), fontsize=20)
    plt.tight_layout()
    ax.set_ylim(-0.2, 0.84)
    ax.grid(True, axis="y", alpha=0.5)
    ax.set_axisbelow(True)      
    plt.show()
    
def significance_bar(start,end,height,displaystring,linewidth = 1.2,
                     markersize = 8,boxpad  =0.3,fontsize = 15,color = 'k'):
    # draw a line with downticks at the ends
    plt.plot([start,end],[height]*2,'-',color = color,lw=linewidth, marker = TICKDOWN,
             markeredgewidth=linewidth, markersize = markersize)
    
    # draw the text with a bounding box covering up the line
    plt.text(0.5*(start+end),height + 0.03, displaystring, ha = 'center',va='center',)
             #bbox=dict(facecolor='1.', edgecolor='none',boxstyle='Square,pad='+str(boxpad)),size = fontsize)
    
def PlotPhenotypeV2(HIQ_Male_dat, HIQ_Female_dat, LIQ_Male_dat, LIQ_Female_dat, REG):
    X = np.arange(HIQ_Male_dat.shape[0])
    fig, ax = plt.subplots(dpi=120, figsize = (8+ HIQ_Male_dat.shape[0]/2, 6))
    ax.bar(X - 0.2, HIQ_Male_dat["EFFECT"], color="#F1948A", width=0.15, label="HIQ LGD", 
           edgecolor ='black')
    ax.bar(X - 0.05, HIQ_Female_dat["EFFECT"], color="#AED6F1", width=0.15, label="HIQ Dmis", 
       edgecolor ='black', hatch="")

    ax.bar(X + 0.1, LIQ_Male_dat["EFFECT"], color="#B03A2E", width=0.15, label="LIQ LGD", 
           edgecolor ='black')
    ax.bar(X + 0.25, LIQ_Female_dat["EFFECT"], color="#21618C", width=0.15, label="LIQ Dmis", 
       edgecolor ='black', hatch="")

    ax.set_xlabel('', fontweight ='bold', fontsize = 20)
    ax.set_ylabel('Expression Bias', fontsize = 20)
    plt.xticks(X, HIQ_Male_dat.index.values, rotation=45, fontsize=10, ha='right', rotation_mode="anchor")

    ax.legend(loc="upper left")
    plt.tight_layout()
    #ax.set_ylim(-0.1, 0.95)
    ax.grid(True, axis="y", alpha=0.5)
    ax.set_axisbelow(True)     
    plt.title(REG)
    plt.show()  
    
def PlotPhenotypeV3(HIQ_Male_dat, HIQ_Male_err, HIQ_Female_dat, HIQ_Female_err, 
                    LIQ_Male_dat, LIQ_Male_err, LIQ_Female_dat, LIQ_Female_err, REG):
    X = np.arange(HIQ_Male_dat.shape[0])
    fig, ax = plt.subplots(dpi=120, figsize = (4+ HIQ_Male_dat.shape[0]/2, 6))
    ax.bar(X - 0.2, HIQ_Male_dat["EFFECT"], yerr = HIQ_Male_err, color="#F1948A", width=0.15, label="HIQ LGD", 
           edgecolor ='black')
    ax.bar(X - 0.05, HIQ_Female_dat["EFFECT"], yerr = HIQ_Female_err, color="#AED6F1", width=0.15, label="HIQ Dmis", 
       edgecolor ='black', hatch="")

    ax.bar(X + 0.1, LIQ_Male_dat["EFFECT"], yerr = LIQ_Male_err, color="#B03A2E", width=0.15, label="LIQ LGD", 
           edgecolor ='black')
    ax.bar(X + 0.25, LIQ_Female_dat["EFFECT"], yerr = LIQ_Female_err, color="#21618C", width=0.15, label="LIQ Dmis", 
       edgecolor ='black', hatch="")

    ax.set_xlabel('', fontweight ='bold', fontsize = 20)
    ax.set_ylabel('Expression Bias', fontsize = 20)
    plt.xticks(X, HIQ_Male_dat.index.values, rotation=45, fontsize=10, ha='right', rotation_mode="anchor")

    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.8))
    plt.tight_layout()
    #ax.set_ylim(-0.1, 0.95)
    ax.grid(True, axis="y", alpha=0.5)
    ax.set_axisbelow(True)     
    plt.title(REG)
    plt.show()  
    
def GetBootsDist_STR(DFs, STR):
    dat = []
    for df in DFs:
        tmp = []
        biases = df.loc[STR, "EFFECT"]
        dat.append(np.mean(biases))
    return np.array(dat)

In [ ]:
HIQ_BootDFs = []
for i in range(1, 1001, 1):
    df1 = pd.read_csv('/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Bootstrap/ASD.HIQ/boot.bias.ALL.{}.csv'.format(i), index_col="STR")
    if "REGION" in df1.columns:
        df1.loc[df1["REGION"] == "Amygdalar", "REGION"] = "Amygdala"
    HIQ_BootDFs.append(df1)

LIQ_BootDFs = []
for i in range(1, 1001, 1):
    df1 = pd.read_csv('/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Bootstrap/ASD.LIQ/boot.bias.ALL.{}.csv'.format(i), index_col="STR")
    if "REGION" in df1.columns:
        df1.loc[df1["REGION"] == "Amygdalar", "REGION"] = "Amygdala"
    LIQ_BootDFs.append(df1)

In [ ]:
HIQ_dat = []
LIQ_dat = []
HIQ_err = []
LIQ_err = []
for REG in CIR_REGIONS:
    HIQ_dat.append(HIQ_Cir[HIQ_Cir["REGION"]==REG]["EFFECT"].mean())
    LIQ_dat.append(LIQ_Cir[LIQ_Cir["REGION"]==REG]["EFFECT"].mean())
    HIQ_err.append(np.std(GetBootsDist(HIQ_BootDFs, CIR_REGIONS_Dict[REG])))
    LIQ_err.append(np.std(GetBootsDist(LIQ_BootDFs, CIR_REGIONS_Dict[REG])))

HIQ_dat = np.array(HIQ_dat)
LIQ_dat = np.array(LIQ_dat)
HIQ_err = np.array(HIQ_err)    
LIQ_err = np.array(LIQ_err)   

In [ ]:
def PlotPhenotype(Dat1, Err1, Dat2, Err2, REG, IQ_PhenotypeDF, x_offset, label1="", label2="", fig_size=0, color=None, ax=None):
    #sort_idx = np.argsort(HighIQ_dat)
    Pvalues = IQ_PhenotypeDF.loc[Dat1.index.values, "Pvalue"].values
    sort_idx = np.argsort(Dat1["EFFECT"].values)
    X = np.arange(Dat1.shape[0])
    if fig_size == 0:
        fig_size = (2.5 + 0.4 * Dat1.shape[0], 6)
    
    # Use provided ax if given, otherwise create new figure
    if ax is None:
        fig, ax = plt.subplots(dpi=120, figsize=fig_size)
    
    dat1 = Dat1["EFFECT"].values[sort_idx]
    err1 = Err1[sort_idx]
    dat2 = Dat2["EFFECT"].values[sort_idx]
    err2 = Err2[sort_idx]
    
    bar_width = 0.4
    
    # Use provided color if specified, otherwise use default colors
    color1 = color if color else "#AED6F1"
    color2 = color if color else "#21618C"
    
    # Calculate x positions based on region index
    region_idx = list(region_colors.keys()).index(REG)
    
    # High IQ bars (left) - solid fill
    ax.bar(X + x_offset - 0.2, dat1, yerr=err1, 
           color=color if color else color1, 
           width=0.4, 
           edgecolor='black')
    
    # Low IQ bars (right) - hatched pattern
    ax.bar(X + x_offset + 0.2, dat2, yerr=err2, 
           color=color if color else color2,
           width=0.4, 
           edgecolor='black',
           hatch='////')  # Add diagonal hatching
    
    Pvalues = Pvalues[sort_idx]
    offset = 0.05
    for i,p in enumerate(Pvalues):
        if p>=0.05:
            continue
        elif p<0.001:
            displaystring = r'***'
        elif p<0.01:
            displaystring = r'**'
        else:
            displaystring = r'*'

        height = offset + max(dat1[i] + err1[i], dat2[i]+err2[i])
        bar_centers = (X[i] + x_offset - 0.4) + np.array([0.5,1.5])*bar_width
        significance_bar(bar_centers[0],bar_centers[1],height,displaystring)
    
    ax.set_xlabel('', fontweight='bold', fontsize=15)
    ax.set_ylabel('Mutation Bias', fontsize=30)
    
    # Add region labels
    labels = drop_fromList(Dat1.index.values[sort_idx])
    ax.set_xticks(X + x_offset)
    ax.set_xticklabels(labels, rotation=45, fontsize=10, ha='right', rotation_mode="anchor")

    # Only add title for individual plots
    if ax is None:
        plt.title(" ".join(REG.split("_")), fontsize=20)
        plt.tight_layout()
        
    ax.set_ylim(-0.2, 0.84)
    ax.grid(True, axis="y", alpha=0.5)
    ax.set_axisbelow(True)
    
    # Increase ytick font size
    ax.tick_params(axis='y', labelsize=25)
    
    # Only show plot immediately for individual plots
    if ax is None:
        plt.show()

# Create figure and axis for combined plot
fig = plt.figure(dpi=100, figsize=(30, 12))
# Create main axis with adjusted position to make room for legends
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])  # [left, bottom, width, height]

# Define colors for each region
region_colors = {'Isocortex': '#268ad5',
'Olfactory_areas': '#5ab4ac',
'Cortical_subplate': '#7ac3fa',
'Hippocampus': '#2c9d39',
'Amygdala': '#742eb5',
'Striatum': '#ed8921',
'Thalamus': '#e82315',
'Midbrain': '#f6b26b',
'Pallidum': '#2ECC71',}

# Create one combined plot
x_offset = 0
all_ticks = []
all_positions = []

for REG in region_colors.keys():
    HIQ_ALL_dat = HIQ_Cir[HIQ_Cir["REGION"]==REG]
    LIQ_ALL_dat = LIQ_Cir[LIQ_Cir["REGION"]==REG]
    REG_STRs = HIQ_ALL_dat.index.values
    HIQ_ALL_err = np.array([np.std(GetBootsDist_STR(HIQ_BootDFs, STR)) for STR in REG_STRs])
    LIQ_ALL_err = np.array([np.std(GetBootsDist_STR(LIQ_BootDFs, STR)) for STR in REG_STRs])
    
    # Store tick positions and labels
    X = np.arange(len(REG_STRs))
    all_positions.extend(X + x_offset)
    all_ticks.extend(drop_fromList(HIQ_ALL_dat.index.values[np.argsort(HIQ_ALL_dat["EFFECT"].values)]))
    
    # Pass ax to PlotPhenotype
    PlotPhenotype(HIQ_ALL_dat, HIQ_ALL_err, LIQ_ALL_dat, LIQ_ALL_err, REG, IQ_PhenotypeDF,
                  x_offset,
                  "Higher IQ Bias", "Lower IQ Bias", 
                  color=region_colors[REG],
                  fig_size=(30,8),
                  ax=ax)
    x_offset += len(REG_STRs)

# Set all tick positions and labels at once
ax.set_xticks(all_positions)
ax.set_xticklabels(all_ticks, rotation=45, ha='right', rotation_mode="anchor", fontsize=25)
ax.tick_params(axis='y', labelsize=25)

# Create legend for IQ groups
legend_ax1 = fig.add_axes([0.85, 0.7, 0.1, 0.2])  # Position for first legend
legend_ax1.axis('off')
solid_patch = plt.Rectangle((0, 0), 1, 1, facecolor='gray', edgecolor='black')
hatched_patch = plt.Rectangle((0, 0), 1, 1, facecolor='gray', hatch='/ / /', edgecolor='black')
legend_ax1.legend([solid_patch, hatched_patch], ['High IQ', 'Low IQ'], fontsize=25)

# Create legend for regions
legend_ax2 = fig.add_axes([0.85, 0.3, 0.1, 0.3])  # Position for second legend
legend_ax2.axis('off')
region_handles = [plt.Rectangle((0, 0), 1, 1, facecolor=color, edgecolor='black') 
                 for color in region_colors.values()]
legend_ax2.legend(region_handles, region_colors.keys(), fontsize=25)

# Create legend for p-values
legend_ax3 = fig.add_axes([0.85, 0.1, 0.1, 0.1])  # Position for third legend
legend_ax3.axis('off')
star1 = plt.Line2D([], [], color='black', marker='*', linestyle='None', markersize=10)
star2 = plt.Line2D([], [], color='black', marker='*', linestyle='None', markersize=10)
star3 = plt.Line2D([], [], color='black', marker='*', linestyle='None', markersize=10)
legend_ax3.legend([star1, star2, star3], ['* p < 0.05', '** p < 0.01', '*** p < 0.001'], fontsize=25)

plt.show()

In [ ]:
def significance_bar(start,end,height,displaystring,linewidth = 1.2,
                     markersize = 8,boxpad  =0.3,fontsize = 15,color = 'k'):
    # draw a line with downticks at the ends
    plt.plot([start,end],[height]*2,'-',color = color,lw=linewidth, marker = TICKDOWN,
             markeredgewidth=linewidth, markersize = markersize)
    
    # draw the text with a bounding box covering up the line
    plt.text(0.5*(start+end),height + 0.03, displaystring, ha = 'center',va='center', fontsize=fontsize)
             #bbox=dict(facecolor='1.', edgecolor='none',boxstyle='Square,pad='+str(boxpad)),size = fontsize)

def PlotPhenotype(Dat1, Err1, Dat2, Err2, REG, IQ_PhenotypeDF, x_offset, label1="", label2="", fig_size=0, color=None, ax=None):
    #sort_idx = np.argsort(HighIQ_dat)
    Pvalues = IQ_PhenotypeDF.loc[Dat1.index.values, "Pvalue"].values
    sort_idx = np.argsort(Dat1["EFFECT"].values)
    X = np.arange(Dat1.shape[0])
    if fig_size == 0:
        fig_size = (2.5 + 0.4 * Dat1.shape[0], 6)
    
    # Use provided ax if given, otherwise create new figure
    if ax is None:
        fig, ax = plt.subplots(dpi=120, figsize=fig_size)
    
    dat1 = Dat1["EFFECT"].values[sort_idx]
    err1 = Err1[sort_idx]
    dat2 = Dat2["EFFECT"].values[sort_idx]
    err2 = Err2[sort_idx]
    
    bar_width = 0.4
    
    # Use provided color if specified, otherwise use default colors
    color1 = color if color else "#AED6F1"
    color2 = color if color else "#21618C"
    
    # Calculate x positions based on region index
    region_idx = list(region_colors.keys()).index(REG)
    
    # High IQ bars (left) - solid fill
    # High IQ bars (left) - filled
    ax.bar(X + x_offset - 0.2, dat1, yerr=err1,
           color='none',  # Transparent fill
           width=0.4,
           edgecolor=color if color else color2,
           linewidth=4)
    
    # Low IQ bars (right) - open/unfilled
    ax.bar(X + x_offset + 0.2, dat2, yerr=err2,
           color=color,  # 
           width=0.4,
           edgecolor=color if color else color2,
           linewidth=1)  # Make edge more visible since bar is unfilled
    
    Pvalues = Pvalues[sort_idx]
    offset = 0.05
    for i,p in enumerate(Pvalues):
        if p>=0.05:
            continue
        elif p<0.001:
            displaystring = r'***'
        elif p<0.01:
            displaystring = r'**'
        else:
            displaystring = r'*'

        height = offset + max(dat1[i] + err1[i], dat2[i]+err2[i])
        bar_centers = (X[i] + x_offset - 0.4) + np.array([0.5,1.5])*bar_width
        significance_bar(bar_centers[0],bar_centers[1],height,displaystring, markersize=20, fontsize=20)
    
    ax.set_xlabel('', fontweight='bold', fontsize=15)
    ax.set_ylabel('Mutation Bias', fontsize=30)
    
    # Add region labels
    labels = drop_fromList(Dat1.index.values[sort_idx])
    ax.set_xticks(X + x_offset)
    ax.set_xticklabels(labels, rotation=45, fontsize=10, ha='right', rotation_mode="anchor")

    # Only add title for individual plots
    if ax is None:
        plt.title(" ".join(REG.split("_")), fontsize=20)
        plt.tight_layout()
        
    ax.set_ylim(-0.2, 0.84)
    ax.grid(True, axis="y", alpha=0.5)
    ax.set_axisbelow(True)
    
    # Increase ytick font size
    ax.tick_params(axis='y', labelsize=25)
    
    # Only show plot immediately for individual plots
    if ax is None:
        plt.show()

# Create figure and axis for combined plot
fig = plt.figure(dpi=100, figsize=(30, 12))
# Create main axis with adjusted position to make room for legends
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])  # [left, bottom, width, height]

# Define colors for each region
region_colors = {'Isocortex': '#268ad5',
'Olfactory_areas': '#5ab4ac',
'Cortical_subplate': '#7ac3fa',
'Hippocampus': '#2c9d39',
'Amygdala': '#742eb5',
'Striatum': '#ed8921',
'Thalamus': '#e82315',
'Midbrain': '#f6b26b',
'Pallidum': '#2ECC71',}

# Create one combined plot
x_offset = 0
all_ticks = []
all_positions = []

for REG in region_colors.keys():
    HIQ_ALL_dat = HIQ_Cir[HIQ_Cir["REGION"]==REG]
    LIQ_ALL_dat = LIQ_Cir[LIQ_Cir["REGION"]==REG]
    REG_STRs = HIQ_ALL_dat.index.values
    HIQ_ALL_err = np.array([np.std(GetBootsDist_STR(HIQ_BootDFs, STR)) for STR in REG_STRs])
    LIQ_ALL_err = np.array([np.std(GetBootsDist_STR(LIQ_BootDFs, STR)) for STR in REG_STRs])
    
    # Store tick positions and labels
    X = np.arange(len(REG_STRs))
    all_positions.extend(X + x_offset)
    all_ticks.extend(drop_fromList(HIQ_ALL_dat.index.values[np.argsort(HIQ_ALL_dat["EFFECT"].values)]))
    
    # Pass ax to PlotPhenotype
    PlotPhenotype(HIQ_ALL_dat, HIQ_ALL_err, LIQ_ALL_dat, LIQ_ALL_err, REG, IQ_PhenotypeDF,
                  x_offset,
                  "Higher IQ Bias", "Lower IQ Bias", 
                  color=region_colors[REG],
                  fig_size=(30,8),
                  ax=ax)
    x_offset += len(REG_STRs)

# Set all tick positions and labels at once
ax.set_xticks(all_positions)
ax.set_xticklabels(all_ticks, rotation=45, ha='right', rotation_mode="anchor", fontsize=25)
ax.tick_params(axis='y', labelsize=25)

# Create legend for IQ groups
legend_ax1 = fig.add_axes([0.85, 0.7, 0.1, 0.2])  # Position for first legend
legend_ax1.axis('off')
filled_patch = plt.Rectangle((0, 0), 1, 1, facecolor='none', edgecolor='black')
open_patch_patch = plt.Rectangle((0, 0), 1, 1, facecolor='gray', edgecolor='black')
legend_ax1.legend([filled_patch, open_patch_patch], ['Higher IQ', 'Lower IQ'], fontsize=25, frameon=False)

# Create legend for regions
legend_ax2 = fig.add_axes([0.88, 0.4, 0.1, 0.3])  # Position for second legend
legend_ax2.axis('off')
region_handles = [plt.Rectangle((0, 0), 1, 1, facecolor=color, edgecolor='black') 
                 for color in region_colors.values()]
legend_ax2.legend(region_handles, region_colors.keys(), fontsize=25, frameon=False)

# Create legend for p-values
legend_ax3 = fig.add_axes([0.85, 0.1, 0.1, 0.1])  # Position for third legend
legend_ax3.axis('off')
star1 = plt.Line2D([], [], color='black', marker='', linestyle='None', markersize=10)
star2 = plt.Line2D([], [], color='black', marker='', linestyle='None', markersize=10)
star3 = plt.Line2D([], [], color='black', marker='', linestyle='None', markersize=10)
legend_ax3.legend([star1, star2, star3], ['* p < 0.05', '** p < 0.01', '*** p < 0.001'], fontsize=25, frameon=False)

plt.show()

In [ ]:
def significance_bar(start,end,height,displaystring,linewidth = 1.2,
                     markersize = 8,boxpad  =0.3,fontsize = 15,color = 'k'):
    # draw a line with downticks at the ends
    plt.plot([start,end],[height]*2,'-',color = color,lw=linewidth, marker = TICKDOWN,
             markeredgewidth=linewidth, markersize = markersize)
    
    # draw the text with a bounding box covering up the line
    plt.text(0.5*(start+end),height + 0.03, displaystring, ha = 'center',va='center', fontsize=fontsize)
             #bbox=dict(facecolor='1.', edgecolor='none',boxstyle='Square,pad='+str(boxpad)),size = fontsize)

def PlotPhenotype(Dat1, Err1, Dat2, Err2, REG, IQ_PhenotypeDF, x_offset, label1="", label2="", fig_size=0, color=None, ax=None):
    Pvalues = IQ_PhenotypeDF.loc[Dat1.index.values, "Pvalue"].values
    sort_idx = np.argsort(Dat1["EFFECT"].values)
    X = np.arange(Dat1.shape[0])
    if fig_size == 0:
        fig_size = (2.5 + 0.4 * Dat1.shape[0], 6)
    
    # Use provided ax if given, otherwise create new figure
    if ax is None:
        fig, ax = plt.subplots(dpi=120, figsize=fig_size)
    
    dat1 = Dat1["EFFECT"].values[sort_idx]
    err1 = Err1[sort_idx]
    dat2 = Dat2["EFFECT"].values[sort_idx]
    err2 = Err2[sort_idx]
    
    # Use provided color if specified, otherwise use default colors
    color1 = color if color else "#AED6F1"
    color2 = color if color else "#21618C"
    
    # Plot dots instead of bars
    # High IQ dots (left)
    ax.errorbar(X + x_offset - 0.2, dat1, yerr=err1,
                fmt='o', markersize=10, color='none',
                markeredgecolor=color if color else color2,
                markeredgewidth=2,
                ecolor=color if color else color2,
                capsize=5)
    
    # Low IQ dots (right)
    ax.errorbar(X + x_offset + 0.2, dat2, yerr=err2,
                fmt='o', markersize=10, color=color,
                markeredgecolor=color if color else color2,
                markeredgewidth=2,
                ecolor=color if color else color2,
                capsize=5)
    
    Pvalues = Pvalues[sort_idx]
    offset = 0.05
    for i,p in enumerate(Pvalues):
        if p>=0.05:
            continue
        elif p<0.001:
            displaystring = r'***'
        elif p<0.01:
            displaystring = r'**'
        else:
            displaystring = r'*'

        height = offset + max(dat1[i] + err1[i], dat2[i]+err2[i])
        dot_centers = (X[i] + x_offset - 0.2, X[i] + x_offset + 0.2)
        significance_bar(dot_centers[0], dot_centers[1], height, displaystring, markersize=20, fontsize=20)
    
    ax.set_xlabel('', fontweight='bold', fontsize=15)
    ax.set_ylabel('Mutation Bias', fontsize=30)
    
    # Add region labels
    labels = drop_fromList(Dat1.index.values[sort_idx])
    ax.set_xticks(X + x_offset)
    ax.set_xticklabels(labels, rotation=45, fontsize=10, ha='right', rotation_mode="anchor")

    # Only add title for individual plots
    if ax is None:
        plt.title(" ".join(REG.split("_")), fontsize=20)
        plt.tight_layout()
        
    ax.set_ylim(-0.2, 0.84)
    ax.grid(True, axis="y", alpha=0.5)
    ax.set_axisbelow(True)
    
    # Increase ytick font size
    ax.tick_params(axis='y', labelsize=25)
    
    # Only show plot immediately for individual plots
    if ax is None:
        plt.show()

# Create figure and axis for combined plot
fig = plt.figure(dpi=100, figsize=(30, 12))
# Create main axis with adjusted position to make room for legends
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])  # [left, bottom, width, height]

# Define colors for each region
region_colors = {'Isocortex': '#268ad5',
'Olfactory_areas': '#5ab4ac',
'Cortical_subplate': '#7ac3fa',
'Hippocampus': '#2c9d39',
'Amygdala': '#742eb5',
'Striatum': '#ed8921',
'Thalamus': '#e82315',
'Midbrain': '#f6b26b',
'Pallidum': '#2ECC71',}

# Create one combined plot
x_offset = 0
all_ticks = []
all_positions = []

for REG in region_colors.keys():
    HIQ_ALL_dat = HIQ_Cir[HIQ_Cir["REGION"]==REG]
    LIQ_ALL_dat = LIQ_Cir[LIQ_Cir["REGION"]==REG]
    REG_STRs = HIQ_ALL_dat.index.values
    HIQ_ALL_err = np.array([np.std(GetBootsDist_STR(HIQ_BootDFs, STR)) for STR in REG_STRs])
    LIQ_ALL_err = np.array([np.std(GetBootsDist_STR(LIQ_BootDFs, STR)) for STR in REG_STRs])
    
    # Store tick positions and labels
    X = np.arange(len(REG_STRs))
    all_positions.extend(X + x_offset)
    all_ticks.extend(drop_fromList(HIQ_ALL_dat.index.values[np.argsort(HIQ_ALL_dat["EFFECT"].values)]))
    
    # Pass ax to PlotPhenotype
    PlotPhenotype(HIQ_ALL_dat, HIQ_ALL_err, LIQ_ALL_dat, LIQ_ALL_err, REG, IQ_PhenotypeDF,
                  x_offset,
                  "Higher IQ Bias", "Lower IQ Bias", 
                  color=region_colors[REG],
                  fig_size=(30,8),
                  ax=ax)
    x_offset += len(REG_STRs)

# Set all tick positions and labels at once
ax.set_xticks(all_positions)
ax.set_xticklabels(all_ticks, rotation=45, ha='right', rotation_mode="anchor", fontsize=25)
ax.tick_params(axis='y', labelsize=25)

# Create legend for IQ groups
legend_ax1 = fig.add_axes([0.85, 0.7, 0.1, 0.2])  # Position for first legend
legend_ax1.axis('off')
filled_dot = plt.Line2D([0], [0], marker='o', color='none', markerfacecolor='none', 
                       markeredgecolor='black', markersize=10, markeredgewidth=2)
open_dot = plt.Line2D([0], [0], marker='o', color='none', markerfacecolor='gray',
                      markeredgecolor='black', markersize=10)
legend_ax1.legend([filled_dot, open_dot], ['Higher IQ', 'Lower IQ'], fontsize=25, frameon=False)

# Create legend for regions
legend_ax2 = fig.add_axes([0.88, 0.4, 0.1, 0.3])  # Position for second legend
legend_ax2.axis('off')
region_handles = [plt.Line2D([0], [0], marker='o', color='none', markerfacecolor=color,
                            markeredgecolor='black', markersize=10)
                 for color in region_colors.values()]
legend_ax2.legend(region_handles, region_colors.keys(), fontsize=25, frameon=False)

# Create legend for p-values
legend_ax3 = fig.add_axes([0.85, 0.1, 0.1, 0.1])  # Position for third legend
legend_ax3.axis('off')
star1 = plt.Line2D([], [], color='black', marker='', linestyle='None', markersize=10)
star2 = plt.Line2D([], [], color='black', marker='', linestyle='None', markersize=10)
star3 = plt.Line2D([], [], color='black', marker='', linestyle='None', markersize=10)
legend_ax3.legend([star1, star2, star3], ['* p < 0.05', '** p < 0.01', '*** p < 0.001'], fontsize=25, frameon=False)

plt.show()

# Sex

In [ ]:
ASC_IQ_dat = pd.read_excel("../doc/1-s2.0-S0092867419313984-mmc4.xlsx", sheet_name="Phenotype")
ASC_IQ_dat = ASC_IQ_dat[ASC_IQ_dat["Role"]=="Proband"]
ASC_IQ_dat = ASC_IQ_dat.dropna(subset=['IQ'])
ASC_Male = ASC_IQ_dat[ASC_IQ_dat["Sex"]=="Male"]["Phenotype_ID"].values
ASC_Female = ASC_IQ_dat[ASC_IQ_dat["Sex"]=="Female"]["Phenotype_ID"].values

Spark_IQ_dat = pd.read_csv("../../ASD_Phenotype/SPARK_Collection_Version6/core_descriptive_variables.csv")
Spark_IQ_dat = Spark_IQ_dat[Spark_IQ_dat["asd"]==True]
Spark_IQ_dat = Spark_IQ_dat.dropna(subset=['fsiq'])
Spark_Male = Spark_IQ_dat[Spark_IQ_dat["sex"]=="Male"]["subject_sp_id"].values
Spark_Female = Spark_IQ_dat[Spark_IQ_dat["sex"]=="Female"]["subject_sp_id"].values

Male = np.concatenate([ASC_Male, Spark_Male])
Female = np.concatenate([ASC_Female, Spark_Female])

ASD_Discov_Muts = pd.read_csv("../dat/genes/SPARK/ASD_Discov_DNVs.txt", delimiter="\t")
ASD_Discov_Trios = pd.read_csv("../dat/genes/SPARK/ASD_Discov_Trios.txt", delimiter="\t")

ASD_Rep_Muts = pd.read_csv("../dat/genes/SPARK/ASD_Rep_DNVs.txt", delimiter="\t")
ASD_Rep_Trios = pd.read_csv("../dat/genes/SPARK/ASD_Rep_Trios.txt", delimiter="\t")

ASD_Muts = pd.concat([ASD_Discov_Muts, ASD_Rep_Muts])
ASD_Trios = pd.concat([ASD_Discov_Trios, ASD_Rep_Trios])

Spark_Meta_2stage = pd.read_excel("../dat/genes/asd/TabS_DenovoWEST_Stage1+2.xlsx",
                           skiprows=2, sheet_name="TopDnEnrich")
Spark_Meta_HC = Spark_Meta_2stage[Spark_Meta_2stage["pDenovoWEST_Meta"]<=1.3e-6]
#Spark_Meta_HC = Spark_Meta_2stage[Spark_Meta_2stage["pDenovoWEST_Meta"]<=1e-03]
HighConfGenes = Spark_Meta_HC["HGNC"].values
HighConfMuts = ASD_Muts[ASD_Muts["HGNC"].isin(HighConfGenes)]
HighConfMuts = Filt_LGD_Mis(HighConfMuts, Dmis=True)

Male_Muts = HighConfMuts[HighConfMuts["IID"].isin(Male)]
Female_Muts = HighConfMuts[HighConfMuts["IID"].isin(Female)]

In [ ]:
Male_Muts.to_csv("../dat/Unionize_bias/Gender.Male.Highconf.Muts.csv", index=False)
Female_Muts.to_csv("../dat/Unionize_bias/Gender.Female.Highconf.Muts.csv", index=False)

In [ ]:
print(Male_Muts.shape, Female_Muts.shape)

In [ ]:
Male_GW = Mut2GeneDF(Male_Muts, LGD=True, Dmis=True) 
Male_GW_LGD = Mut2GeneDF(Male_Muts, LGD=True, Dmis=False) 
Male_GW_Dmis = Mut2GeneDF(Male_Muts, LGD=False, Dmis=True) 
Male_ALL = AvgSTRZ_Weighted(ExpZ2Mat, Male_GW, 
                                 Method = 1, csv_fil = "../dat/Unionize_bias/ASD.Male.ALL.bias.csv")
Male_LGD = AvgSTRZ_Weighted(ExpZ2Mat, Male_GW_LGD, 
                                 Method = 1, csv_fil = "../dat/Unionize_bias/ASD.Male.LGD.bias.csv")
Male_Dmis = AvgSTRZ_Weighted(ExpZ2Mat, Male_GW_Dmis, 
                                 Method = 1, csv_fil = "../dat/Unionize_bias/ASD.Male.Dmis.bias.csv")

Female_GW = Mut2GeneDF(Female_Muts, LGD=True, Dmis=True)
Female_GW_LGD = Mut2GeneDF(Female_Muts, LGD=True, Dmis=False)
Female_GW_Dmis = Mut2GeneDF(Female_Muts, LGD=False, Dmis=True)
Female_ALL = AvgSTRZ_Weighted(ExpZ2Mat, Female_GW, 
                                 Method = 1, csv_fil = "../dat/Unionize_bias/ASD.Female.ALL.bias.csv")
Female_LGD = AvgSTRZ_Weighted(ExpZ2Mat, Female_GW_LGD, 
                                 Method = 1, csv_fil = "../dat/Unionize_bias/ASD.Female.LGD.bias.csv")
Female_Dmis = AvgSTRZ_Weighted(ExpZ2Mat, Female_GW_Dmis, 
                                 Method = 1, csv_fil = "../dat/Unionize_bias/ASD.Female.Dmis.bias.csv")
Male_ALL = Male_ALL.set_index("STR")
Male_LGD = Male_LGD.set_index("STR")
Male_Dmis = Male_Dmis.set_index("STR")
Female_ALL = Female_ALL.set_index("STR")
Female_LGD = Female_LGD.set_index("STR")
Female_Dmis = Female_Dmis.set_index("STR")

In [ ]:
Male_ALL_Cir = Male_ALL.loc[ASD_Circuits,:]
Male_LGD_Cir = Male_LGD.loc[ASD_Circuits,:]
Male_Dmis_Cir = Male_Dmis.loc[ASD_Circuits,:]
Female_ALL_Cir = Female_ALL.loc[ASD_Circuits,:]
Female_LGD_Cir = Female_LGD.loc[ASD_Circuits,:]
Female_Dmis_Cir = Female_Dmis.loc[ASD_Circuits,:]


Male_ALL_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdalar"
Male_LGD_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdalar"
Male_Dmis_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdalar"
Female_ALL_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdalar"
Female_LGD_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdalar"
Female_Dmis_Cir.loc["Bed_nuclei_of_the_stria_terminalis", "REGION"] = "Amygdalar"

In [ ]:
Male_BootDFs = []
Male_BootDFs_LGD = []
Male_BootDFs_Dmis = []
for i in range(1,1001,1):
    df1 = pd.read_csv('../dat/Unionize_bias/Bootstrap/ASD.Male/boot.bias.ALL.{}.csv'.format(i), index_col="STR")
    df2 = pd.read_csv('../dat/Unionize_bias/Bootstrap/ASD.Male/boot.bias.LGD.{}.csv'.format(i), index_col="STR")
    df3 = pd.read_csv('../dat/Unionize_bias/Bootstrap/ASD.Male/boot.bias.Dmis.{}.csv'.format(i), index_col="STR")
    Male_BootDFs.append(df1)
    Male_BootDFs_LGD.append(df2)
    Male_BootDFs_Dmis.append(df3)

Female_BootDFs = []
Female_BootDFs_LGD = []
Female_BootDFs_Dmis = []
for i in range(1,1001,1):
    df1 = pd.read_csv('../dat/Unionize_bias/Bootstrap/ASD.Female/boot.bias.ALL.{}.csv'.format(i), index_col="STR")
    df2 = pd.read_csv('../dat/Unionize_bias/Bootstrap/ASD.Female/boot.bias.LGD.{}.csv'.format(i), index_col="STR")
    df3 = pd.read_csv('../dat/Unionize_bias/Bootstrap/ASD.Female/boot.bias.Dmis.{}.csv'.format(i), index_col="STR")
    Female_BootDFs.append(df1)
    Female_BootDFs_LGD.append(df2)
    Female_BootDFs_Dmis.append(df3)

In [ ]:
CIR_REGIONS = ['Isocortex', 'Hippocampus', 'Amygdalar', 'Striatum', 'Thalamus', 'Olfactory_areas']
Male_ALL_dat = []
Male_LGD_dat = []
Male_Dmis_dat = []
Female_ALL_dat = []
Female_LGD_dat = []
Female_Dmis_dat = []

Male_err = []
Male_LGD_err = []
Male_Dmis_err = []
Female_err = []
Female_LGD_err = []
Female_Dmis_err = []

for REG in CIR_REGIONS:
    Male_ALL_dat.append(Male_ALL_Cir[Male_ALL_Cir["REGION"]==REG]["EFFECT"].mean())
    Male_LGD_dat.append(Male_LGD_Cir[Male_LGD_Cir["REGION"]==REG]["EFFECT"].mean())
    Male_Dmis_dat.append(Male_Dmis_Cir[Male_Dmis_Cir["REGION"]==REG]["EFFECT"].mean())
    Female_ALL_dat.append(Female_ALL_Cir[Female_ALL_Cir["REGION"]==REG]["EFFECT"].mean())
    Female_LGD_dat.append(Female_LGD_Cir[Female_LGD_Cir["REGION"]==REG]["EFFECT"].mean())
    Female_Dmis_dat.append(Female_Dmis_Cir[Female_Dmis_Cir["REGION"]==REG]["EFFECT"].mean())
    
    Male_err.append(np.std(GetBootsDist(Male_BootDFs, CIR_REGIONS_Dict[REG])))
    Male_LGD_err.append(np.std(GetBootsDist(Male_BootDFs_LGD, CIR_REGIONS_Dict[REG])))
    Male_Dmis_err.append(np.std(GetBootsDist(Male_BootDFs_Dmis, CIR_REGIONS_Dict[REG])))
    Female_err.append(np.std(GetBootsDist(Female_BootDFs, CIR_REGIONS_Dict[REG])))
    Female_LGD_err.append(np.std(GetBootsDist(Female_BootDFs_LGD, CIR_REGIONS_Dict[REG])))
    Female_Dmis_err.append(np.std(GetBootsDist(Female_BootDFs_Dmis, CIR_REGIONS_Dict[REG])))
    
Male_ALL_dat = np.array(Male_ALL_dat)
Male_LGD_dat = np.array(Male_LGD_dat)
Male_Dmis_dat = np.array(Male_Dmis_dat)    
Female_ALL_dat = np.array(Female_ALL_dat)
Female_LGD_dat = np.array(Female_LGD_dat)
Female_Dmis_dat = np.array(Female_Dmis_dat)  
Male_err = np.array(Male_err)
Male_LGD_err = np.array(Male_LGD_err)
Male_Dmis_err = np.array(Male_Dmis_err)
Female_err = np.array(Female_err)
Female_LGD_err = np.array(Female_LGD_err)
Female_Dmis_err = np.array(Female_Dmis_err)

In [ ]:

CIR_REGIONS__ = np.array(CIR_REGIONS)
fig, ax = plt.subplots(dpi=480)

X = np.arange(len(CIR_REGIONS))

ax.bar(X - 0.1, Male_ALL_dat[sort_idx], yerr=Male_err[sort_idx], color="#e31a1c", width=0.2, label="Male", 
       edgecolor ='black')
ax.bar(X + 0.1, Female_ALL_dat[sort_idx], yerr=Female_err[sort_idx], color="#ff7f00", width=0.2, label="Female", 
   edgecolor ='black', hatch="")

ax.set_xlabel('', fontweight ='bold', fontsize = 20)
ax.set_ylabel('Mutation Bias', fontsize = 20)
plt.xticks(X, CIR_REGIONS__[sort_idx], rotation=45, fontsize=10, ha='right', rotation_mode="anchor")
  
ax.legend(loc="upper left")
plt.tight_layout()
#ax.set_ylim(0, 0.65)
ax.grid(True, axis="y", alpha=0.5)
ax.set_axisbelow(True)      
#plt.title("Bias - ALL")